In [ ]:
import pandas as pd
from dotenv import load_dotenv
from time import time as ttt
from tqdm import tqdm
load_dotenv()
from openai import OpenAI
client = OpenAI()

from bert_score import score
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from transformers import AutoTokenizer
from IPython.display import display
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("model")
tokenizer.model_max_length = 1000000000

In [ ]:
df = pd.read_csv('testdata.csv',encoding='utf-8')
df.head(2)

## gpt 3.5

In [ ]:
def gen_prompt(question):
    return [
        {"role": "user", "content": question}
    ]

def ask_gpt(
    question,
    model="gpt-3.5-turbo-0125",
):
    try:
        ttti = ttt()
        messages = gen_prompt(question)
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            # default settings
            temperature=0,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            # stop=stop_sequence,
        )
        tttf = ttt()
        return [True, response.choices[0].message.content, tttf-ttti]
    except Exception as e:
        return [False, e, 0.0]

In [ ]:
answers = []
times = []

for index, row in tqdm(df.iterrows()):
    question = row['question']
    status, response, time = ask_gpt(question)
    if status:
        answers.append(response)
        times.append(time)
    else:
        print(f"{index} failed: {response}")
        answers.append(None)
        times.append(None)

df['response'] = answers
df['time'] = times

In [ ]:
df.head(2)

In [ ]:
df.to_csv('e2e-gpt.csv',index=False,encoding='utf-8')

In [ ]:
df = pd.read_csv('e2e/e2e-gpt.csv',encoding='utf-8')
print(len(df))
df['time'].mean()

## ours time

In [ ]:
ir = pd.read_csv("ir/ir-ir.csv",encoding='utf-8')["irt"].to_list()
ft = pd.read_csv("fttrt/trt-ft.csv",encoding='utf-8')["time_finetune"].to_list()

## bert score gpt

In [ ]:
model_type = 'microsoft/deberta-xlarge-mnli'  

labels = df["answer"].to_list()
b = df["response"].to_list()

print('inference gpt-3.5')
P, R, F1 = score(b, labels, model_type=model_type, verbose=False)
print('precision\t', P.mean().item())
print('recall\t', R.mean().item())
print('f1\t', F1.mean().item())

## rouge score gpt

In [ ]:
labels = df["answer"].to_list()
bs = df["response"].to_list()

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True, tokenizer=tokenizer)
fb = [0,0,0]

for label, b in zip(labels, bs):
    scores = scorer.score(label, b)
    fb[0] += scores['rouge1'].fmeasure
    fb[1] += scores['rouge2'].fmeasure
    fb[2] += scores['rougeL'].fmeasure

for i in range(3):
    fb[i] /= len(labels)

fb

## bert score tanoy

In [ ]:
for index, row in df.iterrows():
    if row["time"]==300:
        df.loc[index, "time"] = 60

In [ ]:
# df = pd.read_csv('e2e/e2e-tanoy.csv',encoding='utf-8')
print(len(df))
df['time'].mean()

In [ ]:
model_type = 'microsoft/deberta-xlarge-mnli'  

labels = df["answer"].to_list()
b = df["response_clean"].to_list()

print('inference tanoy')
P, R, F1 = score(b, labels, model_type=model_type, verbose=False)
print('precision\t', P.mean().item())
print('recall\t', R.mean().item())
print('f1\t', F1.mean().item())

## rouge score tanoy

In [ ]:
labels = df["answer"].to_list()
bs = df["response_clean"].to_list()

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True, tokenizer=tokenizer)
fb = [0,0,0]

for label, b in zip(labels, bs):
    scores = scorer.score(label, b)
    fb[0] += scores['rouge1'].fmeasure
    fb[1] += scores['rouge2'].fmeasure
    fb[2] += scores['rougeL'].fmeasure

for i in range(3):
    fb[i] /= len(labels)

fb